In [1]:
from google.cloud import storage
import os

In [44]:
# ローカルのみ
from oauth2client.client import GoogleCredentials
# cred = GoogleCredentials.get_application_default()
# cred = GoogleCredentials.from_json('./cyberagent-143-47faf9af3ea2.json')

In [3]:
from utils import to_var, idx2word, interpolate, AttributeDict
from utils import words2input, ids2ptext

In [4]:
import json

In [5]:
import torch
from model_kwbase import SentenceVAE
from ptb import SOS_INDEX, UNK_INDEX, PAD_INDEX, EOS_INDEX

In [ ]:
func_name = 'kawamoto-dews-exp-20191220-kw2title-hr-mediatag'
! gcloud functions deploy kawamoto-dews-exp-20191220-kw2title-hr-mediatag --trigger-http --runtime=python37

In [31]:
# データのダウンロード
bucket_name = 'kawamoto-ramiel'
data_name='hr.mediatag'
model_url = f'experiments_v1_kw2copy_20191220/models/{data_name}/E9.pytorch'
meta_url = f'experiments_v1_kw2copy_20191220/models/{data_name}/model_meta.json'

src_vocab_url = f'experiments_v1_kw2copy_20191220/data/{data_name}/src/ptb.vocab.json'
tgt_vocab_url = f'experiments_v1_kw2copy_20191220/data/{data_name}/tgt/ptb.vocab.json'

model_filename = 'model.pytorch'
meta_filename = 'model_meta.json'
src_vocab_filename = 'src.vocab.json'
tgt_vocab_filename = 'tgt.vocab.json'

save_dir = './tmp'
model_path = os.path.join(save_dir, model_filename)
meta_path = os.path.join(save_dir, meta_filename)
src_vocab_path = os.path.join(save_dir, src_vocab_filename)
tgt_vocab_path = os.path.join(save_dir, tgt_vocab_filename)

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

/Users/s07309/.local/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/s07309/.local/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/auth

In [11]:
def load_json(path):
    return json.load(open(path, 'r'))

In [12]:
def download_gs_data(url, save_path):
    blob = bucket.blob(url)
    blob.download_to_filename(save_path)
    
# モデルの読み込み
def load_model(path, meta_path):
    with open(meta_path) as f:
        meta_dict = AttributeDict(json.load(f))
    
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu')
        
    state_dict = torch.load(path, map_location=device)
    model_shapes = {k: v.shape for k,v in state_dict.items()}
    ext_kwargs = {}
    
    # BOW Loss
    bow_hidden_shape = model_shapes.get('latent2bow.0.weight')
    use_bow_loss = bow_hidden_shape is not None
    print(f'BOW Loss: {use_bow_loss}')
    if use_bow_loss:
        ext_kwargs['bow_hidden_size'] = bow_hidden_shape[0]
    else:
        ext_kwargs['use_bow_loss'] = False
        
    # Latent size
    latent_size = model_shapes.get('hidden2logv.bias')[0]
    print(f'Latent size: {latent_size}')
    
    # Gumbel
    gumbel_vocab_size, gumbel_embedding_size = model_shapes.get('hidden2gumbel.weight', [None, None])
    is_gumbel = gumbel_vocab_size is not None
    print(f'Gumbel: {is_gumbel}')
    if is_gumbel:
        ext_kwargs['is_gumbel'] = is_gumbel
        ext_kwargs['gumbel_tau'] = tau_dict[name]
    print(ext_kwargs)
    
    vocab_size = model_shapes['embedding.weight'][0]
    
    # KW Base
    out_vocab_size, _ = model_shapes.get('decoder_embedding.weight', [None, None])
    if out_vocab_size is not None:
        ext_kwargs['out_vocab_size'] = out_vocab_size
        
    model = SentenceVAE(
        vocab_size=vocab_size,
        sos_idx=SOS_INDEX,
        eos_idx=EOS_INDEX,
        pad_idx=PAD_INDEX,
        unk_idx=UNK_INDEX,
        max_sequence_length=meta_dict.max_sequence_length,
        embedding_size=meta_dict.embedding_size,
        rnn_type=meta_dict.rnn_type,
        hidden_size=meta_dict.hidden_size,
        word_dropout=meta_dict.word_dropout,
        embedding_dropout=meta_dict.embedding_dropout,
        latent_size=latent_size,
        num_layers=meta_dict.num_layers,
        bidirectional=meta_dict.bidirectional,        
        **ext_kwargs,
    )
    print(model)
    
    if not os.path.exists(path):
        raise FileNotFoundError(path)

    model.load_state_dict(state_dict)
    print("Model loaded from %s"%(path))

    if torch.cuda.is_available():
        model = model.cuda()
        
    model.eval()
    return model


def sample_z_from_gaussian(mean, logv):
    std = torch.exp(0.5 * logv)
    z = to_var(torch.randn([logv.shape[1]]))
    z = z * std + mean
    return z


def encode_z_list(model, sample_input, sample_length, n):
    z_list = []
    with torch.no_grad():
        hidden = model.encode(sample_input, sample_length)
        mean, logv, z = model.hidden2latent(hidden)
        z_list.append(z)
        for _ in range(n-1):
            z = sample_z_from_gaussian(mean, logv)
            z_list.append(z)
    return torch.cat(z_list)


# 推論関数
def inference(word_list, model, src_vocab, tgt_vocab, n=10):
    # Inputへ整形
    sample = words2input(word_list, src_vocab['w2i'])
    sample_input, sample_length = sample['input'], sample['length']
    # Encode
    z_list = encode_z_list(model, sample_input, sample_length, n)
    # Decode
    generations, _ = model.inference(z = z_list)
    # Outputへ整形
    decode_texts = [ids2ptext(g, tgt_vocab['i2w']) for g in generations]
    return decode_texts

In [14]:
def main(request):
    # データのダウンロード
    if not os.path.exists(model_path):
        download_gs_data(model_url, model_path)
        download_gs_data(src_vocab_url, src_vocab_path)
        download_gs_data(tgt_vocab_url, tgt_vocab_path)
        download_gs_data(meta_url, meta_path)
        
    # インスタンス化
    global model, src_vocab, tgt_vocab
    if model is None or src_vocab is None or tgt_vocab is None:
        model = load_model(model_path, meta_path)
        src_vocab, tgt_vocab = load_json(src_vocab_path), load_json(tgt_vocab_path)        
        
    if not (request.args and 'keyword' in  request.args):
        return jsonify({'message': 'no keyword'}), 500
    
    keyword_list = request.args.getlist('keyword')
    n = request.args.get('n', 10)
    
    decoded_texts = inference(word_list, model, src_vocab, tgt_vocab, n=n)
    
    return  jsonify({'message': 'success', 'decoded': decoded_texts}), 200

In [ ]:
if __name__ == "__main__":
    from flask import Flask, request
    app = Flask(__name__)

    @app.route('/')
    def index():
        return main(request)

    app.run('127.0.0.1', 8000, debug=True)

In [28]:
src_vocab_url

'experiments_v1_kw2copy_20191220/data/hr.mediatag/src/ptb.vocab.json'

In [29]:
tgt_vocab_url

'experiments_v1_kw2copy_20191220/data/hr.mediatag/tgt/ptb.vocab.json'

In [30]:
meta_url

'experiments_v1_kw2copy_20191220/models/{data_name}/model_meta.json'

In [34]:
%%time
download_gs_data(model_url, model_path)
download_gs_data(src_vocab_url, src_vocab_path)
download_gs_data(tgt_vocab_url, tgt_vocab_path)
download_gs_data(meta_url, meta_path)

CPU times: user 202 ms, sys: 67.5 ms, total: 269 ms
Wall time: 1 s


In [35]:
model = load_model(model_path, meta_path)
src_vocab, tgt_vocab = load_json(src_vocab_path), load_json(tgt_vocab_path)        

BOW Loss: False
Latent size: 16
Gumbel: False
{'use_bow_loss': False}
SentenceVAE(
  (embedding): Embedding(1235, 300)
  (embedding_dropout): Dropout(p=0.5, inplace=False)
  (decoder_embedding): Embedding(5622, 300)
  (encoder_rnn): GRU(300, 256, batch_first=True)
  (decoder_rnn): GRU(300, 256, batch_first=True)
  (hidden2mean): Linear(in_features=256, out_features=16, bias=True)
  (hidden2logv): Linear(in_features=256, out_features=16, bias=True)
  (latent2hidden): Linear(in_features=16, out_features=256, bias=True)
  (outputs2vocab): Linear(in_features=256, out_features=5622, bias=True)
)
Model loaded from ./tmp/model.pytorch


In [41]:
word_list = ['看護師', '日勤', '月']
decoded_texts = inference(word_list, model, src_vocab, tgt_vocab, n=30)

In [42]:
decoded_texts

['【急募】看護師、月<num>万、日勤のみ',
 '看護師、<num>月入職、月給<num>万円、日勤のみ',
 '【高待遇の歯科医師求人】業界最大級の案件数、業界最大級!',
 '【急募】看護師、月給<num>万、日勤のみ',
 '【高待遇にて募集中!】未経験歓迎の求人<num>,<num>件以上!いま、大手・優良企業が採用を強化しています。',
 '看護師、<num>月入職、月給<num>万、日勤のみ',
 '【急募】看護師、月収<num>万、賞与<num>ヶ月',
 '看護師、<num>月入職、月給<num>万円、日勤のみ',
 '【急募】看護師、月<num>万、日勤のみ',
 '【急募】保育士の求人サイト',
 '【介護福祉士限定】月給<num>万円/年休<num>日. . .好待遇の求人が増加中',
 '[理学療法士]<num>歳からの転職で年収<num>万円・賞与<num>回',
 '【急募】看護師、月給<num>万、日勤のみ',
 '【急募】看護師、月<num>万、日勤のみ',
 '【急募】看護師、月収<num>万円以上の求人多数!',
 '看護師、<num>月入職、月給<num>万、日勤のみ',
 '看護師、<num>月入職、月給<num>万、日勤のみ',
 '【<num>月入社の求人特集】',
 '【急募】保育士、月給<num>万、日勤のみ',
 '【flexy】<num>年連続支持率<num>%以下!「イースix」を公開!猛牛vs猛牛!',
 '看護師、<num>月入職、月給<num>万円、日勤のみ',
 '【介護福祉士限定】月給<num>万円/年休<num>日. . .好待遇の求人が多数!',
 '【急募】看護師、月<num>日、日勤のみ',
 '【急募】看護師、月収<num>万、土日休みなど、女性に人気の求人が増加中',
 '【急募】看護師、月<num>万、日勤のみ',
 '【<num>年連続顧客満足度no.<num>】月給<num>万/週休<num>日の介護現場に転職',
 '【急募】看護師、月<num>万、日勤のみ',
 '【急募】<num>年連続支持率no<num>のお店があなたのスキルを活かして、新しい挑戦をしませんか?',
 '【介護福祉士限定】月給<num>万、<num>時まで/週休<num>日が当たり前',
 '【急募】看護師